In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Source

Census Household Pulse Survey (HPS): https://www.census.gov/data/experimental-data-products/household-pulse-survey.html

Original data.gov source
https://catalog.data.gov/dataset/mental-health-care-in-the-last-4-weeks
Contains summary data, not disaggregated data. But hints at interesting data and questions in that data

data.gov points to this source
https://www.cdc.gov/nchs/covid19/pulse/mental-health-care.htm
Interesting, but does not have the actual data. Does have a summary report

Searching gets you back to the Census and the detailed data for the Household Pulse Survey
https://www.census.gov/data/experimental-data-products/household-pulse-survey.html


## Survey and Survey Data Summary

- Public use files (PUFs) vs data tables
- Lots of work needed to understand the files and the weights
- Census data is much more complicated than I realized
- Timeframe of CPS vs other surveys
 

## Survey Weights

- Lots of work needed to understand specific weightings
- Person vs household vs replicate weights
- Examples of when to use weights vs when to not
- Talk about how adjusting with weights might actually reduce performance for this project
- This is one of the main areas to work on further or get help from domain experts for future iterations
- While unsure here, we can still move forward in a way that defaults to leaving things as they are (unweighted) until sure that you need to adjust based on weights

## Data Summary

- Which phase(s) are we using and why?
- Are we looking to limit the feature space?
- What type of preprocessing do we expect to do
- Talk about what data comes with the PUFs
- PUFs vs data tables

## Project

- Why unsupervised is useful here
- What unsupervised allows us to do that supervised does not
- Emphasize the speed and messiness of this data coming in compared to other surveys
- Talk about how we will account for representation issues through possibly anomaly detection or another technical approach
- Can use unsupervised output to then return to demographic data to try and provide more useful info

## References

Household Pulse Survey technical documentation: 
- https://www.census.gov/programs-surveys/household-pulse-survey/technical-documentation.html

Info about survey weights:
- https://www.pewresearch.org/methods/2018/01/26/how-different-weighting-methods-work/
- https://pages.nyu.edu/jackson/design.of.social.research/Readings/Johnson%20-%20Introduction%20to%20survey%20weights%20%28PRI%20version%29.pdf
- https://analythical.com/blog/weighting-data-explained

Replicate weights for the Census Current Population Survey (CPS):
- https://cps.ipums.org/cps/repwt.shtml
- 
R example of working with CPS data and replicate weights:
- https://www.adambibler.com/post/exploring-census-household-pulse-survey-part-1/

Articles about using CPS data and weights:
- https://www.jchs.harvard.edu/blog/using-the-census-bureaus-household-pulse-survey-to-assess-the-economic-impacts-of-covid-19-on-americas-households
- https://www.bgsu.edu/content/dam/BGSU/college-of-arts-and-sciences/center-for-family-and-demographic-research/documents/Workshops/2020-Household-Pulse-Survey.pdf

Put in links to where to find supplementary PDFs for HPS and Census data
- https://www2.census.gov/programs-surveys/demo/technical-documentation/hhp/2020_HPS_Background.pd-f
- https://www2.census.gov/programs-surveys/demo/technical-documentation/hhp/2020_HPS_NR_Bias_Report-final.pdf
- https://www.census.gov/content/dam/Census/library/publications/2010/acs/Chapter_11_RevisedDec2010.pdf
- https://www.census.gov/content/dam/Census/programs-surveys/ahs/tech-documentation/2015/Quick_Guide_to_Estimating_Variance_Using_Replicate_Weights_2009_to_Current.pdf
- https://www2.census.gov/programs-surveys/cps/datasets/2018/supp/PERSON-level_Use_of_the_Public_Use_Replicate_Weight_File.doc
- https://www2.census.gov/programs-surveys/cps/datasets/2021/march/Guidance_on_Using_Replicate_Weights_2020-2021.pdf

Video presentations that include chunks about the HPS or other Census data: 
- https://www.youtube.com/watch?v=ltyT34S3C90
- https://www.youtube.com/watch?v=lLk6esuBI6M
- https://www.youtube.com/watch?v=aJDQmsmCv7A
- https://www.youtube.com/watch?v=zfXohiWjzzY
- https://www.youtube.com/watch?v=5Re4D1Ht74k